# 中古音對應閩南語文音查詢系統
### 後續程式

以下要作的事情，就是想辦法以最工整的方式歸納出<<廣韻>>的聲韻與閩南語讀音的對應關係，作出精簡的圖表，並且最終作以streamlit的方式呈現。

In [1]:
import requests
import re
import numpy as np
import time
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import matplotlib
import matplotlib.font_manager
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import streamlit as st

# font setting
#!gdown 1lv0-HBUhnM0rg7rOWDYWPUp93WB5xwsM -O taipei_sans_tc_beta.ttf
matplotlib.font_manager.fontManager.addfont('taipei_sans_tc_beta.ttf')
matplotlib.rc('font', family = 'Taipei Sans TC Beta')

In [659]:
'''# 韻典網/廣韻/小韻表
blk = 'https://ytenx.org'
tulk = 'https://ytenx.org/kyonh/sieux'
tulk2 = ''

Gtdf0 = pd.DataFrame()
Gtndf0 = pd.DataFrame(columns=['字', '小韻'])

Gtu = requests.get(tulk)


while Gtu.status_code == 200:
    
    # 提取小韻表
    Gtsp = BeautifulSoup(Gtu.text, 'html.parser')
    Gttb = Gtsp.find_all('table')[0]
    Gtdf= pd.read_html(StringIO(str(Gttb)))[0]

    # 尋找每個小韻中的字
    for i in Gttb.find_all('tr'):
        if i.find_all('td') != []:
            href = i.find_all('td')[1].find('a').get('href')
        else:
            continue
        Gtun = requests.get(blk+href)
        if Gtun.status_code == 200:
            Gtnsp = BeautifulSoup(Gtun.text, 'html.parser')
            Gtntb = Gtnsp.find_all('table')[-1]
            Gtndf= pd.DataFrame(pd.read_html(StringIO(str(Gtntb)))[0][0])
            Gtndf.columns = ['字']
            Gtndf['小韻'] = i.find_all('td')[1].find('a').text
        Gtndf0 = pd.concat([Gtndf0,Gtndf],ignore_index=True) 

    # 整理小韻表
    Gtdf0 = pd.concat([Gtdf0,Gtdf],ignore_index=True)

    # 尋找下一頁
    tulk2 = BeautifulSoup(Gtu.text, 'html.parser').find_all('ul')[-1].find_all('a')[-1].get('href')  
    if tulk2 == '':
        break
    time.sleep(1)
    Gtu = requests.get(tulk+tulk2)
else:
    print("請求失敗")

# join
G = pd.merge(Gtndf0, Gtdf0, on='小韻', how='inner')
G.head(20)

# 移除除了聲、韻、調以外的欄位
del G['次序']
del G['反切']
    
# 當小韻是破音字而重複使用時，會有重複的字
G = G.drop_duplicates() 

# 將DataFrame保存為CSV檔
G.to_csv('sieux.csv', index=False)
G.tail(20)

# 59m 41.5s
'''

,字,小韻,聲母,韻目,調,等,呼,韻母
26544,䁆,腌,影,業,入,三,開,業
26545,殜,殜,以,業,入,三,開,業
26546,𩐱,殜,以,業,入,三,開,業
26547,跲,跲,羣,業,入,三,開,業
26548,㭘,跲,羣,業,入,三,開,業
26549,昅,跲,羣,業,入,三,開,業
26550,极,跲,羣,業,入,三,開,業
26551,笈,跲,羣,業,入,三,開,業
26552,乏,乏,並,乏,入,三,合,乏
26553,泛,乏,並,乏,入,三,合,乏


In [167]:
'''# 教育部閩南語常用字典
Blk = 'https://sutian.moe.edu.tw/'
mnlk = 'https://sutian.moe.edu.tw/zh-hant/siannuntiau/'
href = re.sub(rf'{Blk}', '', mnlk)

kaotian = pd.DataFrame()

Mn = requests.get(mnlk)
while requests.get(Blk+href).status_code== 200:
    Mnsp = BeautifulSoup(Mn.text, 'html.parser')
    Mnul = Mnsp.find_all('ul')[-2]

    # 依調>韻母>聲母順序搜尋
    for i in Mnul.find_all('a')[:-1]:
        # 抓出調
        tiao = re.findall(r'\d',i.text)[0]

        # 搜尋同調之韻母
        href = i.get('href')
        Mnul2 = requests.get(Blk+href)
        Mnsp2 = BeautifulSoup(Mnul2.text, 'html.parser')
        Mnul3 = Mnsp2.find_all('ul')[4]
            
        for j in Mnul3.find_all('a'):
            # 抓出韻母
            un = re.sub(r'\W','', j.text)

            # 搜尋同韻母之聲母
            href = j.get('href')
            Mntr = requests.get(Blk+href)
            Mntr = BeautifulSoup(Mntr.text, 'html.parser')
            Mntr = Mntr.find_all('tr')

            for k in Mntr:
                # 抓出聲母
                if k.find_all('td') == []:
                    continue
                siann = re.sub(rf'{un}', '', k.find_all('td')[0].text)
                # 抓出字
                Mndf = pd.DataFrame([i.text for i in k.find_all('a')], columns = ['字'])

                # 找出文白屬性
                wb = []
                for l in k.find_all('a'):
                    href = l.get('href')
                    Mna = requests.get(Blk+href)
                    Mna = BeautifulSoup(Mna.text, 'html.parser')
                    Mna = Mna.find('table').find('span', class_="tiongkuahho-giapa")
                    if Mna == None:
                        wb.append('')
                        continue
                    wb.append(Mna.text)
                    
                Mndf['閩南語聲母'] = siann
                Mndf['閩南語韻母'] = un
                Mndf['閩南語調'] = tiao
                Mndf['文白音'] = wb
                kaotian = pd.concat([kaotian,Mndf],ignore_index=True)

                time.sleep(1)

        print(f'第{tiao}調完成')

    break
else:
    print("請求失敗")

# 消除聲調記號
kaotian['閩南語韻母'] = [re.sub(r'á|à|â|ā|a̍', 'a', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'é|è|ê|ē|e̍', 'e', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'í|ì|î|ī|i̍', 'i', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ó|ò|ô|ō|o̍', 'o', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ú|ù|û|ū|u̍', 'u', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ḿ|m̀|m̂|m̄|m̍', 'm', i) for i in kaotian['閩南語韻母']]
kaotian['閩南語韻母'] = [re.sub(r'ń|ǹ|n̂|n̄|n̍', 'n', i) for i in kaotian['閩南語韻母']]

# 將DataFrame保存為CSV檔
kaotian.to_csv('kaotian.csv', index=False)
kaotian.tail(20)
'''
# 約三小時


第1調完成
第2調完成
第3調完成
第4調完成
第5調完成
第7調完成
第8調完成


,字,閩南語聲母,閩南語韻母,閩南語調,文白音
6300,罰,h,uat,8,
6301,劣,l,uat,8,
6302,拔,p,uat,8,文
6303,菝,p,uat,8,替
6304,跋,p,uat,8,
6305,鈸,p,uat,8,文
6306,絕,ts,uat,8,文
6307,奪,t,uat,8,
6308,悅,,uat,8,
6309,曰,,uat,8,


In [2]:
G = pd.read_csv('./sieux.csv', encoding='utf-8')
kaotian = pd.read_csv('./kaotian.csv', encoding='utf-8')

# 調整資料
# 增加AB的項目
G['AB'] = [re.findall(r'A|B', i) for i in G['韻母']]
G['AB'] = [i[0] if i != [] else '-' for i in G['AB']]
# 統一不同聲調的韻目
G['韻目'] = [re.sub(r'一|二|三|開|合|A|B', '', i) if i != '合' else '合' for i in G['韻母']]

# fillna
kaotian = kaotian.fillna('-')

# 合併閩南語字典與廣韻
D = pd.merge(G, kaotian, on='字', how='inner')
D.head(20)

,字,小韻,聲母,韻目,調,等,呼,韻母,AB,閩南語聲母,閩南語韻母,閩南語調,文白音
0,東,東,端,東,平,一,開,東一,-,t,ang,1,白
1,東,東,端,東,平,一,開,東一,-,t,ong,1,文
2,凍,東,端,東,平,一,開,東一,-,t,ang,3,白
3,凍,東,端,東,平,一,開,東一,-,t,ong,3,文
4,凍,涷,端,東,去,一,開,東一,-,t,ang,3,白
5,凍,涷,端,東,去,一,開,東一,-,t,ong,3,文
6,同,同,定,東,平,一,開,東一,-,t,ang,5,白
7,同,同,定,東,平,一,開,東一,-,t,ong,5,文
8,仝,同,定,東,平,一,開,東一,-,k,ang,7,替
9,仝,全,從,仙,平,三,合,仙合,-,k,ang,7,替


In [7]:
bun = D[D['文白音'] == '文']
b_siann = bun.groupby(['聲母', '閩南語聲母']).size().reset_index(name='字數')

# 使用 transform 計算比例並新增一列 '比例'
b_siann['比例'] = b_siann['字數'] / b_siann.groupby('聲母')['字數'].transform('sum')
b_siann = b_siann.sort_values(by='比例', ascending=False)

# 剛才已經由高到低排好中古聲母與閩南語各相應關係的比例，愈前面的比例愈大，代表愈有可能是對應的聲母
# 廣韻裡面的破音字多到嚇死人，平均每4個字就有1個是破音字。那些頻率低的相應關係可能只是沒有對到相應的讀音
# 所以我們從最前面開始，依次將該聲母與所對應最合理的閩南語聲母標記為 True，並將其餘不同的讀音標記為 False，然後刪除
bun['temp'] = False
for i, row in b_siann.iterrows():
    s = row['聲母']
    ms = row['閩南語聲母']

    for j in list(bun[(bun['聲母'] == s) & (bun['閩南語聲母'] == ms)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['聲母'] == s) & (bun['閩南語聲母'] == ms) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 破音字清乾淨之後，中古聲母與閩南語各相應關係應該會有不小改變，所以我們再重新計算一次比例
b_siann = bun.groupby(['聲母', '閩南語聲母']).size().reset_index(name='字數')
b_siann['比例'] = b_siann['字數'] / b_siann.groupby('聲母')['字數'].transform('sum')
b_siann = b_siann.sort_values(by='比例', ascending=False)
b_siann['比例'] = round(b_siann['比例'], 2)

# 每個對應關係中，挑出一個例字，能夠理解得更清楚
b_siann['例字'] = [list(bun[(bun['聲母'] == row['聲母']) & (bun['閩南語聲母'] == row['閩南語聲母'])]['字'])[0] for i, row in b_siann.iterrows()]
bg_siann = pd.DataFrame(b_siann.groupby(['聲母', '閩南語聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bg_siann.head(20)


C:\Users\O\AppData\Local\Temp\ipykernel_10060\1081574936.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bun['temp'] = False


比例 例字
聲母 閩南語聲母         
並  h      0.32  篷
   p      0.53  盤
   ph     0.15  皮
云  -      0.93  芋
   h      0.07  雄
以  -      0.97  溶
   l      0.03  簷
來  l      0.99  籠
   n      0.01  賴
初  tsh    1.00  差
匣  -      0.02  盒
   h      0.96  洪
   k      0.02  缸
孃  l      1.00  鐃
定  t      0.85  同
   th     0.15  苔
崇  s      0.25  事
   ts     0.50  狀
   tsh    0.25  床
常  s      0.96  垂

In [8]:
# 由於似乎沒找到甚麼有效的方法可以畫出三層以上的群組長條圖，所以以下只有最基本的中古音與閩南語聲韻調的對應關係會以圖呈現，等呼的差異只能以表格呈現

# 聲母，第一張圖
x = [
    b_siann.sort_values('聲母').iloc[0:39]['聲母'],
    b_siann.sort_values('聲母').iloc[0:39]['閩南語聲母']
]
figs1 = go.Figure()
# 把資料分成上下兩張圖，不然會太擠
figs1.add_bar(x=x,y=b_siann.sort_values('聲母')[0:39]['比例'], text=b_siann.sort_values('聲母')[0:39]['例字']
            , textposition='outside', textfont=dict(size=9))

# 變更顏色、添加標題和座標軸名稱
figs1.update_layout(
    colorway=['#FFA15A', '#636EFA'],
    title='閩南語聲母對應比例 (上)',
    xaxis_title='聲母',
    yaxis_title='比例'
)

figs1.show()

''''''

# 聲母，第二張圖
x = [
    b_siann.sort_values('聲母').iloc[40:79]['聲母'],
    b_siann.sort_values('聲母').iloc[40:79]['閩南語聲母']
]
figs2 = go.Figure()
figs2.add_bar(x=x,y=b_siann.sort_values('聲母')[40:79]['比例'], text=b_siann.sort_values('聲母')[40:79]['例字']
            , textposition='outside', textfont=dict(size=9))

# 變更顏色、添加標題和座標軸名稱
figs2.update_layout(
    colorway=['#FFA15A', '#636EFA'],
    title='閩南語聲母對應比例 (下)',
    xaxis_title='聲母',
    yaxis_title='比例'
)

figs2.show()

In [9]:
# 找出中古聲母對應閩南語聲母有分歧的聲母，分析看看是為甚麼
# 先假設閩南語聲母的分歧是因為等的差異，所以我們先找出不同的等
l = [i for i in b_siann['聲母'] if (b_siann[b_siann['聲母'] == i]['閩南語聲母'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['聲母'] == i]['等'].unique()) != 1)]
bst0 = bun.query('聲母 in @l')
bst = bst0.groupby(['聲母', '等', '閩南語聲母']).size().reset_index(name='字數')
bst['比例'] = bst['字數'] / bst.groupby(['聲母', '等'])['字數'].transform('sum')

''''''
# 消除破音字，跟之前一樣
bst = bst.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in bst.iterrows():
    s = row['聲母']
    ms = row['閩南語聲母']
    t = row['等']

    for j in list(bun[(bun['聲母'] == s) & (bun['閩南語聲母'] == ms) & (bun['等'] == t)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['聲母'] == s) & (bun['閩南語聲母'] == ms) & (bun['等'] == t) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
bst0 = bun.query('聲母 in @l')
bst = bst0.groupby(['聲母', '等', '閩南語聲母']).size().reset_index(name='字數')
bst['比例'] = bst['字數'] / bst.groupby(['聲母', '等'])['字數'].transform('sum')
bst['比例'] = round(bst['比例'], 2)

''''''

# 聲母的分歧不見得都與等相關，因此比較一個字對應到同樣的閩南語聲母，不同等的標準差，並將標準差大於0.2才的聲母挑出來，來粗略鑑驗其相關性
for i in bst['聲母'].unique():
    tnum = len(bun[bun['聲母'] == i]['等'].unique())
    npstd = 0
    for j in bst[bst['聲母'] == i]['閩南語聲母'].unique():
        l = list(bst[(bst['聲母'] == i) & (bst['閩南語聲母'] == j)]['比例'])
        while len(l) < tnum:
            l.append(0)
        npstd += np.std(l)*(bst[(bst['聲母'] == i) & (bst['閩南語聲母'] == j)].shape[0] / bst[bst['聲母'] == i].shape[0])
    #print(i, npstd)
    bst.loc[bst['聲母'] == i,'temp'] = npstd
# 若標準差未大於0.2，則認為分歧不夠，恐怕不是等的問題，刪除。以下皆以標準差大於0.2為準
bst = bst[bst['temp'] > 0.2]
del bst['temp']

''''''
# 例字
bst['例字'] = [list(bst0[(bst0['聲母'] == row['聲母']) & (bst0['閩南語聲母'] == row['閩南語聲母']) & (bst0['等'] == row['等'])]['字'])[0] for i, row in bst.iterrows()]
bgst = pd.DataFrame(bst.groupby(['聲母','等', '閩南語聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bgst


比例 例字
聲母 等 閩南語聲母         
並  一 h      0.06  篷
     p      0.56  盤
     ph     0.38  磐
   三 h      0.58  縫
     p      0.35  便
     ph     0.06  皮
   二 p      0.86  琶
     ph     0.14  彭
   四 p      1.00  並
幫  一 p      1.00  般
   三 h      0.69  楓
     p      0.31  鞭
   二 p      1.00  飽
   四 p      1.00  邊
明  一 b      0.65  門
     m      0.35  梅
   三 b      1.00  夢
   二 b      0.11  猛
     m      0.89  茅
   四 b      1.00  眠
滂  一 ph     1.00  潘
   三 h      0.62  豐
     ph     0.38  篇
   二 ph     1.00  樸
   四 ph     1.00  批
疑  一 g      0.33  外
     ng     0.67  鵝
   三 g      1.00  魚
   二 -      0.25  瓦
     g      0.62  牙
     ng     0.12  雅
   四 g      1.00  研

In [3]:
bun = D[D['文白音'] == '文']
b_un = bun.groupby(['韻目', '閩南語韻母']).size().reset_index(name='字數')

# 使用 transform 計算比例並新增一列 '比例'
b_un['比例'] = b_un['字數'] / b_un.groupby('韻目')['字數'].transform('sum')
b_un = b_un.sort_values(by='比例', ascending=False)
bun['temp'] = False

# 消除破音字
for i, row in b_un.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
b_un = bun.groupby(['韻目', '閩南語韻母']).size().reset_index(name='字數')
b_un['比例'] = b_un['字數'] / b_un.groupby('韻目')['字數'].transform('sum')
b_un = b_un.sort_values(by='比例', ascending=False)
b_un['比例'] = round(b_un['比例'], 2)

# 例字
b_un['例字'] = [list(bun[(bun['韻目'] == row['韻目']) & (bun['閩南語韻母'] == row['閩南語韻母'])]['字'])[0] for i, row in b_un.iterrows()]
bg_un = pd.DataFrame(b_un.groupby(['韻目', '閩南語韻母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bg_un.head(20)


C:\Users\O\AppData\Local\Temp\ipykernel_21632\2056279569.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bun['temp'] = False


比例 例字
韻目 閩南語韻母         
之  i      0.15  基
   it     0.04  值
   u      0.81  思
乏  uat    1.00  乏
仙  ian    0.68  鮮
   uan    0.32  全
佳  ai     0.90  鞋
   ua     0.10  掛
侯  io     0.22  歐
   o      0.04  透
   oo     0.74  侯
侵  im     1.00  尋
元  ian    0.24  軒
   uan    0.76  園
先  ian    1.00  先
冬  ong    1.00  冬
凡  uan    1.00  帆
刪  an     0.20  般
   uan    0.80  關
合  ap     1.00  合

In [4]:
# 韻母，第一張圖
x = [
    b_un.sort_values('韻目').iloc[0:47]['韻目'],
    b_un.sort_values('韻目').iloc[0:47]['閩南語韻母']
]
figu1 = go.Figure()
# 把資料分成三張圖，不然會太擠
figu1.add_bar(x=x,y=b_un.sort_values('韻目')[0:47]['比例'], text=b_un.sort_values('韻目')[0:47]['例字']
            , textposition='outside', textfont=dict(size=9))

# 變更顏色、添加標題和座標軸名稱
figu1.update_layout(
    colorway=['#D25350', '#636EFA'],
    title='閩南語韻母對應比例 (上)',
    xaxis_title='韻目',
    yaxis_title='比例'
)

figu1.show()

''''''

# 韻母，第二張圖
x = [
    b_un.sort_values('韻目').iloc[48:96]['韻目'],
    b_un.sort_values('韻目').iloc[48:96]['閩南語韻母']
]
figu2 = go.Figure()
figu2.add_bar(x=x,y=b_un.sort_values('韻目')[48:96]['比例'], text=b_un.sort_values('韻目')[48:96]['例字']
            , textposition='outside', textfont=dict(size=9))

# 變更顏色、添加標題和座標軸名稱
figu2.update_layout(
    colorway=['#D25350', '#636EFA'],
    title='閩南語韻母對應比例 (中)',
    xaxis_title='韻目',
    yaxis_title='比例'
)

figu2.show()

''''''

# 韻母，第三張圖
x = [
    b_un.sort_values('韻目').iloc[97:]['韻目'],
    b_un.sort_values('韻目').iloc[97:]['閩南語韻母']
]
figu3 = go.Figure()
figu3.add_bar(x=x,y=b_un.sort_values('韻目')[97:]['比例'], text=b_un.sort_values('韻目')[97:]['例字']
            , textposition='outside', textfont=dict(size=9))

# 變更顏色、添加標題和座標軸名稱
figu3.update_layout(
    colorway=['#D25350', '#636EFA'],
    title='閩南語韻母對應比例 (下)',
    xaxis_title='韻目',
    yaxis_title='比例'
)

# 第三張圖
figu3.show()


In [5]:
# 找出分歧韻母，分析開合口呼的因素
l = [i for i in b_un['韻目'] if (b_un[b_un['韻目'] == i]['閩南語韻母'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['韻目'] == i]['呼'].unique()) != 1)]
buh0 = bun.query('韻目 in @l')
buh = buh0.groupby(['韻目', '呼', '閩南語韻母']).size().reset_index(name='字數')
buh['比例'] = buh['字數'] / buh.groupby(['韻目', '呼'])['字數'].transform('sum')

''''''

# 消除破音字
buh = buh.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in buh.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']
    h = row['呼']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
buh0 = bun.query('韻目 in @l')
buh = buh0.groupby(['韻目', '呼', '閩南語韻母']).size().reset_index(name='字數')
buh['比例'] = buh['字數'] / buh.groupby(['韻目', '呼'])['字數'].transform('sum')
buh['比例'] = round(buh['比例'], 2)

''''''

# 鑑驗相關性
for i in buh['韻目'].unique():
    npstd = 0
    for j in buh[buh['韻目'] == i]['閩南語韻母'].unique():
        l = list(buh[(buh['韻目'] == i) & (buh['閩南語韻母'] == j)]['比例'])
        while len(l) < 2:
            l.append(0)
        npstd += np.std(l)*(buh[(buh['韻目'] == i) & (buh['閩南語韻母'] == j)].shape[0] / buh[buh['韻目'] == i].shape[0])
    print(i, npstd)
    buh.loc[buh['韻目'] == i, 'temp'] = npstd
buh = buh[buh['temp'] > 0.2]
del buh['temp']

''''''

# 例字
buh['例字'] = [list(buh0[(buh0['韻目'] == row['韻目']) & (buh0['閩南語韻母'] == row['閩南語韻母']) & (buh0['呼'] == row['呼'])]['字'])[0] for i, row in buh.iterrows()]
bguh = pd.DataFrame(buh.groupby(['韻目','呼', '閩南語韻母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bguh


仙 0.5
佳 0.5
元 0.5
刪 0.5
唐 0.015000000000000008
微 0.33499999999999996
支 0.5
泰 0.5
清 0.25
皆 0.5
祭 0.3333333333333333
職 0.05999999999999999
脂 0.25
薛 0.5
陽 0.34125
麻 0.25125


比例 例字
韻目 呼 閩南語韻母         
仙  合 uan    1.00  全
   開 ian    1.00  鮮
佳  合 ua     1.00  掛
   開 ai     1.00  鞋
元  合 uan    1.00  園
   開 ian    1.00  軒
刪  合 uan    1.00  關
   開 an     1.00  般
微  合 i      0.33  未
     ui     0.67  飛
   開 i      1.00  幾
支  合 ui     1.00  垂
   開 i      1.00  支
泰  合 ue     1.00  外
   開 ai     1.00  蓋
清  合 ing    0.50  營
     iong   0.50  瓊
   開 ing    1.00  清
皆  合 uai    1.00  怪
   開 ai     1.00  齋
祭  合 ue     0.50  歲
     ui     0.50  脆
   開 e      1.00  誓
脂  合 ui     1.00  龜
   開 e      0.09  梨
     i      0.36  眉
     u      0.55  師
薛  合 uat    1.00  說
   開 iat    1.00  折
陽  合 ong    1.00  坊
   開 iang   0.05  涼
     iong   0.86  陽
     ong    0.09  瘡
麻  合 ua     1.00  花
   開 a      0.84  差
     e      0.03  砂
     ia     0.14  蛇

In [6]:
# 找出分析開合口呼後還有分歧的韻母，分析等的因素
l = [i for i in buh['韻目'] if (buh[(buh['韻目'] == i)]['比例'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['韻目'] == i]['等'].unique()) != 1)]
buth0 = bun.query('韻目 in @l')
buth = buth0.groupby(['韻目', '等', '呼', '閩南語韻母']).size().reset_index(name='字數')
buth['比例'] = buth['字數'] / buth.groupby(['韻目', '等', '呼'])['字數'].transform('sum')

''''''

# 消除破音字
buth = buth.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in buth.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']
    t = row['等']
    h = row['呼']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h) & (bun['等'] == t)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h) & (bun['等'] == t) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
buth0 = bun.query('韻目 in @l')
buth = buth0.groupby(['韻目', '等', '呼', '閩南語韻母']).size().reset_index(name='字數')
buth['比例'] = buth['字數'] / buth.groupby(['韻目', '等', '呼'])['字數'].transform('sum')
buth['比例'] = round(buth['比例'], 2)

''''''

# 鑑驗相關性
for i in buth['韻目'].unique():
    tnum = len(bun[bun['韻目'] == i]['等'].unique())
    npstd = 0
    for j in buth[buth['韻目'] == i]['閩南語韻母'].unique():
        l = list(buth[(buth['韻目'] == i) & (buth['閩南語韻母'] == j)]['比例'])
        while len(l) < tnum:
            l.append(0)
        npstd += np.std(l)*(buth[(buth['韻目'] == i) & (buth['閩南語韻母'] == j)].shape[0] / buth[buth['韻目'] == i].shape[0])
    print(i, npstd)
    buth.loc[buth['韻目'] == i, 'temp'] = npstd
buth = buth[buth['temp'] > 0.2]
del buth['temp']

''''''

# 例字
buth['例字'] = [list(buth0[(buth0['韻目'] == row['韻目']) & (buth0['閩南語韻母'] == row['閩南語韻母']) & (buth0['等'] == row['等']) & (buth0['呼'] == row['呼'])]['字'])[0] for i, row in buth.iterrows()]
bguth = pd.DataFrame(buth.groupby(['韻目','等', '呼', '閩南語韻母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bguth


麻 0.37499999999999994


比例 例字
韻目 等 呼 閩南語韻母         
麻  三 開 ia     1.00  蛇
   二 合 ua     1.00  花
     開 a      0.97  差
       e      0.03  砂

In [12]:
# 找出分析等呼後還有分歧的韻母，分析聲母的因素
l = [i for i in buth['韻目'] if (buth[(buth['韻目'] == i)]['比例'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['韻目'] == i]['等'].unique()) != 1)]
buths0 = bun.query('韻目 in @l')
buths = buths0.groupby(['韻目', '等', '呼', '聲母', '閩南語韻母']).size().reset_index(name='字數')
buths['比例'] = buths['字數'] / buths.groupby(['韻目', '等', '呼', '閩南語韻母'])['字數'].transform('sum')

''''''

# 消除破音字
buths = buths.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in buths.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']
    t = row['等']
    h = row['呼']
    s = row['聲母']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h) & (bun['等'] == t) & (bun['聲母'] == s)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h) & (bun['等'] == t) & (bun['聲母'] == s) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
buths0 = bun.query('韻目 in @l')
buths = buths0.groupby(['韻目', '等', '呼', '聲母', '閩南語韻母']).size().reset_index(name='字數')

# 這裡在做的事是將同一個韻目、閩南語韻母組合對應的聲母合併
# 先是把聲母組用list統整起來，套到每一個字的聲母欄位
buths['聲母'] = [list(buths0[(buths0['韻目'] == row['韻目']) & (buths0['閩南語韻母'] == row['閩南語韻母']) & (buths0['呼'] == row['呼']) & (buths0['等'] == row['等'])]['聲母']) for i, row in buths.iterrows()]
# 接著要把list裡的聲母拆成一個字串
def O(x):
    o = ''
    x = list(set(x))
    for i in list(x):
        o += str(i)
    return o
buths['聲母'] = buths['聲母'].apply(O)
buths = buths.drop_duplicates()
# 重新計算比例
buths['比例'] = buths.groupby(['韻目', '閩南語韻母','呼', '等'])['字數'].transform('sum') / buths.groupby(['韻目','呼', '等'])['字數'].transform('sum')
#buths['比例'] = buths['字數'] / buths.groupby(['韻目', '等', '呼', '閩南語韻母'])['字數'].transform('sum')
buths['比例'] = round(buths['比例'], 2)

'''

# 鑑驗相關性
for i in buths['韻目'].unique():
    munum = len(bun[bun['韻目'] == i]['閩南語韻母'].unique())
    npstd = 0
    for j in buths[buths['韻目'] == i]['聲母'].unique():
        l = list(buths[(buths['韻目'] == i) & (buths['聲母'] == j)]['比例'])
        while len(l) < munum:
            l.append(0)
        npstd += np.std(l)*(buths[(buths['韻目'] == i) & (buths['聲母'] == j)].shape[0] / buths[buths['韻目'] == i].shape[0])
    print(i, npstd)
    buths.loc[buths['韻目'] == i, 'temp'] = npstd
buths = buths[buths['temp'] > 0.2]
del buths['temp']

'''

# 例字
buths['例字'] = [list(buths0[(buths0['韻目'] == row['韻目']) & (buths0['閩南語韻母'] == row['閩南語韻母']) & (buths0['等'] == row['等']) & (buths0['呼'] == row['呼'])]['字'])[0] for i, row in buths.iterrows()]
# 沒有分歧的部分挑出來
buths = buths[buths['比例'] != 1]
bguth = pd.DataFrame(buths.groupby(['韻目','等', '呼', '閩南語韻母', '聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bguth


比例 例字
韻目 等 呼 閩南語韻母 聲母                
麻  二 開 a     影生疑並明見幫初匣  0.95  差
       e     生          0.05  砂

In [14]:
# 找出分析與呼無關的韻母，分析等的因素
l = [i for i in b_un['韻目'] if (b_un[(b_un['韻目'] == i)]['比例'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['韻目'] == i]['等'].unique()) != 1)]
but0 = bun.query('(韻目 in @l) & ~(韻目 in @buh["韻目"])')
but = but0.groupby(['韻目', '等', '閩南語韻母']).size().reset_index(name='字數')
but['比例'] = but['字數'] / but.groupby(['韻目', '等', '閩南語韻母'])['字數'].transform('sum')

''''''

# 消除破音字
but = but.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in but.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']
    t = row['等']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['等'] == t)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['等'] == t) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
but0 = bun.query('韻目 in @l & ~(韻目 in @buh["韻目"])')
but = but0.groupby(['韻目', '等', '閩南語韻母']).size().reset_index(name='字數')
but['比例'] = but['字數'] / but.groupby(['韻目', '等'])['字數'].transform('sum')
but['比例'] = round(but['比例'], 2)

''''''

# 鑑驗相關性
for i in but['韻目'].unique():
    tnum = len(bun[bun['韻目'] == i]['等'].unique())
    npstd = 0
    for j in but[but['韻目'] == i]['閩南語韻母'].unique():
        l = list(but[(but['韻目'] == i) & (but['閩南語韻母'] == j)]['比例'])
        while len(l) < tnum:
            l.append(0)
        npstd += np.std(l)*(but[(but['韻目'] == i) & (but['閩南語韻母'] == j)].shape[0] / but[but['韻目'] == i].shape[0])
    print(i, npstd)
    but.loc[but['韻目'] == i, 'temp'] = npstd
but = but[but['temp'] > 0.2]
del but['temp']

''''''

# 例字
but['例字'] = [list(but0[(but0['韻目'] == row['韻目']) & (but0['閩南語韻母'] == row['閩南語韻母']) & (but0['等'] == row['等'])]['字'])[0] for i, row in but.iterrows()]
bgut = pd.DataFrame(but.groupby(['韻目','等', '閩南語韻母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bgut


屋 0.39499999999999996
東 0.355


比例 例字
韻目 等 閩南語韻母         
屋  一 ok     1.00  曝
   三 iok    0.79  畜
     ok     0.21  覆
東  一 ong    1.00  東
   三 iong   0.71  中
     ong    0.29  夢

In [29]:
# 找出分析呼後還有分歧且與呼無關的韻母，看看其對應的聲母有哪些
l = [i for i in b_un['韻目'] if (buh[(buh['韻目'] == i)]['比例'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['韻目'] == i]['聲母'].unique()) != 1)]
buhs0 = bun.query('(韻目 in @l) & (韻目 in @buh["韻目"]) & ~(韻目 in @buth["韻目"]) & ~(韻目 in @but["韻目"])')
buhs = buhs0.groupby(['韻目', '呼', '聲母', '閩南語韻母']).size().reset_index(name='字數')
buhs['比例'] = buhs.groupby(['韻目', '閩南語韻母', '呼'])['字數'].transform('sum') / buhs.groupby(['韻目', '呼'])['字數'].transform('sum')

''''''

# 消除破音字
buhs = buhs.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in buhs.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']
    h = row['呼']
    s = row['聲母']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h) & (bun['聲母'] == s)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['呼'] == h) & (bun['聲母'] == s) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
buhs0 = bun.query('(韻目 in @l) & (韻目 in @buh["韻目"]) & ~(韻目 in @buth["韻目"]) & ~(韻目 in @but["韻目"])')
buhs = buhs0.groupby(['韻目', '呼', '聲母', '閩南語韻母']).size().reset_index(name='字數')

# 統整聲母
buhs['聲母'] = [list(buhs0[(buhs0['韻目'] == row['韻目']) & (buhs0['閩南語韻母'] == row['閩南語韻母']) & (buhs0['呼'] == row['呼'])]['聲母']) for i, row in buhs.iterrows()]
# 把list拆成str
def O(x):
    o = ''
    x = list(set(x))
    for i in list(x):
        o += str(i)
    return o
buhs['聲母'] = buhs['聲母'].apply(O)
buhs = buhs.drop_duplicates()
# 重新計算比例
buhs['比例'] = buhs.groupby(['韻目', '閩南語韻母', '呼'])['字數'].transform('sum') / buhs.groupby(['韻目', '呼'])['字數'].transform('sum')
buhs['比例'] = round(buhs['比例'], 2)

''''''

# 例字
buhs['例字'] = [list(buhs0[(buhs0['韻目'] == row['韻目']) & (buhs0['閩南語韻母'] == row['閩南語韻母']) & (buhs0['呼'] == row['呼'])]['字'])[0] for i, row in buhs.iterrows()]
# 沒有分歧的部分挑出來
buhs = buhs[buhs['比例'] != 1]
bguhs = pd.DataFrame(buhs.groupby(['韻目','呼', '閩南語韻母', '聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bguhs


比例 例字
韻目 呼 閩南語韻母 聲母                         
微  合 i     明                   0.50  未
     ui    並幫                  0.50  飛
清  合 ing   以                   0.50  營
     iong  羣                   0.50  瓊
祭  合 ue    心                   0.50  歲
     ui    清                   0.50  脆
脂  開 e     來                   0.17  梨
     i     來章定明                0.17  眉
     u     生書心                 0.67  師
陽  開 iang  來澄                  0.08  涼
     iong  日影澄崇曉精邪以孃昌章見心來書清知常  0.72  陽
     ong   崇生初莊                0.19  瘡

In [17]:
# 找出分析等後還有分歧且與呼無關的韻母，看看其對應的聲母有哪些
l = [i for i in b_un['韻目'] if (but[(but['韻目'] == i)]['比例'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['韻目'] == i]['聲母'].unique()) != 1)]
buts0 = bun.query('(韻目 in @l) & (韻目 in @but["韻目"]) & ~(韻目 in @buh["韻目"])')
buts = buts0.groupby(['韻目', '等', '聲母', '閩南語韻母']).size().reset_index(name='字數')
buts['比例'] = buts.groupby(['韻目', '閩南語韻母', '等'])['字數'].transform('sum') / buts.groupby(['韻目', '等'])['字數'].transform('sum')

''''''

# 消除破音字
buts = buts.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in buts.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']
    t = row['等']
    s = row['聲母']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['等'] == t) & (bun['聲母'] == s)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['等'] == t) & (bun['聲母'] == s) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
buts0 = bun.query('韻目 in @l & (韻目 in @but["韻目"])  & ~(韻目 in @buh["韻目"])')
buts = buts0.groupby(['韻目', '等', '聲母', '閩南語韻母']).size().reset_index(name='字數')

# 統整聲母
buts['聲母'] = [list(buts0[(buts0['韻目'] == row['韻目']) & (buts0['閩南語韻母'] == row['閩南語韻母']) & (buts0['等'] == row['等'])]['聲母']) for i, row in buts.iterrows()]
# 把list拆成str
def O(x):
    o = ''
    x = list(set(x))
    for i in list(x):
        o += str(i)
    return o
buts['聲母'] = buts['聲母'].apply(O)
buts = buts.drop_duplicates()
# 重新計算比例
buts['比例'] = buts.groupby(['韻目', '閩南語韻母', '等'])['字數'].transform('sum') / buts.groupby(['韻目', '等'])['字數'].transform('sum')
buts['比例'] = round(buts['比例'], 2)

''''''

# 例字
buts['例字'] = [list(buts0[(buts0['韻目'] == row['韻目']) & (buts0['閩南語韻母'] == row['閩南語韻母']) & (buts0['等'] == row['等'])]['字'])[0] for i, row in buts.iterrows()]
# 沒有分歧的部分挑出來
buts = buts[buts['比例'] != 1]
bguts = pd.DataFrame(buts.groupby(['韻目','等', '閩南語韻母', '聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bguts


比例 例字
韻目 等 閩南語韻母 聲母                
屋  三 iok   日生知徹心見來書澄常  0.5  畜
     ok    滂明幫         0.5  覆
東  三 iong  云澄知見羣       0.7  中
     ong   滂明幫         0.3  夢

In [19]:
# 找出有分歧且與等呼無關的韻母，看看其對應的聲母有哪些
l = [i for i in b_un['韻目'] if (b_un[b_un['韻目'] == i]['閩南語韻母'].count() != 1)]
l = list(set(l))
l = [i for i in l if (len(bun[bun['韻目'] == i]['聲母'].unique()) != 1)]
bus0 = bun.query('(韻目 in @l) & ~(韻目 in @buh["韻目"]) & ~(韻目 in @buts["韻目"])')
bus = bus0.groupby(['韻目', '聲母', '閩南語韻母']).size().reset_index(name='字數')
bus['比例'] = bus.groupby(['韻目', '閩南語韻母'])['字數'].transform('sum') / bus.groupby(['韻目'])['字數'].transform('sum')

''''''

# 消除破音字
bus = bus.sort_values(by='比例', ascending=False)
bun['temp'] = False
for i, row in bus.iterrows():
    u = row['韻目']
    mu = row['閩南語韻母']
    s = row['聲母']

    for j in list(bun[(bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['聲母'] == s)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['韻目'] == u) & (bun['閩南語韻母'] == mu) & (bun['聲母'] == h) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
bus0 = bun.query('(韻目 in @l) & ~(韻目 in @buh["韻目"]) & ~(韻目 in @buts["韻目"])')
bus = bus0.groupby(['韻目', '聲母', '閩南語韻母']).size().reset_index(name='字數')

# 統整聲母
bus['聲母'] = [list(bus0[(bus0['韻目'] == row['韻目']) & (bus0['閩南語韻母'] == row['閩南語韻母'])]['聲母']) for i, row in bus.iterrows()]
# 把list拆成str
def O(x):
    o = ''
    x = list(set(x))
    for i in list(x):
        o += str(i)
    return o
bus['聲母'] = bus['聲母'].apply(O)
bus = bus.drop_duplicates()
# 重新計算比例
bus['比例'] = bus.groupby(['韻目', '閩南語韻母'])['字數'].transform('sum') / bus.groupby(['韻目'])['字數'].transform('sum')
bus['比例'] = round(bus['比例'], 2)

# 例字
bus['例字'] = [list(bus0[(bus0['韻目'] == row['韻目']) & (bus0['閩南語韻母'] == row['閩南語韻母'])]['字'])[0] for i, row in bus.iterrows()]
bgus = pd.DataFrame(bus.groupby(['韻目', '閩南語韻母', '聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
bgus


比例 例字
韻目 閩南語韻母 聲母                     
之  i     日澄見             0.14  基
   it    澄               0.05  值
   u     崇生精邪以心書莊常       0.81  思
侯  io    溪明影             0.21  歐
   o     透               0.07  透
   oo    定影精見來端匣         0.71  侯
唐  iong  影               0.04  映
   ong   定曉溪並精明透見心來幫清端匣  0.96  湯
尤  iu    云疑溪羣孃見來書知       0.77  留
   u     並幫              0.23  浮
德  ik    端明從             0.50  得
   ok    幫               0.50  北
戈  o     定曉溪從滂明見匣        0.75  過
   onn   曉               0.12  火
   oo    來               0.12  螺
文  uan   云               0.14  員
   un    云並明幫            0.86  墳
歌  o     定影溪見來匣          0.50  歌
   oo    疑               0.50  鵝
江  ang   見               0.33  講
   iong  溪               0.33  腔
   ong   匣見              0.33  扛
洽  ap    初               0.50  插
   iap   匣               0.50  狹
灰  ue    曉從              0.33  悔
   ui    明透              0.67  推
燭  ik    疑               0.50  獄
   iok   疑溪以昌章心來清        0.50  燭
盍  ap    來透              0.75  蠟
   iap   來               0.25  臘
眞  im    日               0.33  忍
   in    澄精章來書清知常        0.50  振
   un    澄               0.17  陣
職  ik    云生以船心來書         0.75  力
   it    船               0.25  食
蒸  im    見               0.12  矜
   ing   影並船書知常          0.88  承
藥  iat   心               0.25  削
   iok   日澄疑精以見來知        0.75  著
覺  ak    匣               0.33  學
   ok    影澄滂幫知           0.33  樸
   oo    生               0.33  數
豪  au    並               0.09  抱
   o     定並泥心來清端         0.55  牢
   onn   曉               0.18  好
   oo    明               0.18  毛
質  ik    來               0.50  栗
   it    明船澄             0.50  實
銜  am    見               0.80  監
   an    生               0.20  芟
鍾  iong  羣邪以從徹章見來清澄      0.91  鍾
   ong   並幫              0.09  封
魚  oo    生初              0.30  初
   u     曉疑知以邪見書澄常       0.70  魚
齊  e     定影從透心見          0.75  臍
   i     滂端              0.25  批

In [23]:
# 彙整所有的歧音與其聲母的對應關係
BGUS = bus.query('~(韻目 in @buts["韻目"])')
BGUS = pd.concat([BGUS, buts],ignore_index=True)
BGUS = BGUS.query('~(韻目 in @buths["韻目"])')
BGUS = pd.concat([BGUS, buths],ignore_index=True)
BGUS = BGUS.query('~(韻目 in @buhs["韻目"])')
BGUS = pd.concat([BGUS, buhs],ignore_index=True)
# 以下的原則是這樣：沒有區別就不需要寫出屬性，所以空出來的位置都可以'-'替代
BGUS = BGUS.fillna('-')

BGUS = pd.DataFrame(BGUS.groupby(['韻目','等','呼', '閩南語韻母', '聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
BGUS.head(20)

比例 例字
韻目 等 呼 閩南語韻母 聲母                     
之  - - i     日澄見             0.14  基
       it    澄               0.05  值
       u     崇生精邪以心書莊常       0.81  思
侯  - - io    溪明影             0.21  歐
       o     透               0.07  透
       oo    定影精見來端匣         0.71  侯
唐  - - iong  影               0.04  映
       ong   定曉溪並精明透見心來幫清端匣  0.96  湯
尤  - - iu    云疑溪羣孃見來書知       0.77  留
       u     並幫              0.23  浮
屋  三 - iok   日生知徹心見來書澄常      0.50  畜
       ok    滂明幫             0.50  覆
微  - 合 i     明               0.50  未
       ui    並幫              0.50  飛
德  - - ik    端明從             0.50  得
       ok    幫               0.50  北
戈  - - o     定曉溪從滂明見匣        0.75  過
       onn   曉               0.12  火
       oo    來               0.12  螺
文  - - uan   云               0.14  員

In [19]:
# 彙整所有的韻母對應關係
BGUN = b_un.query('~(韻目 in @buh["韻目"])')
BGUN = pd.concat([BGUN, buh],ignore_index=True)
BGUN = BGUN.query('~(韻目 in @buth["韻目"])')
BGUN = pd.concat([BGUN, buth],ignore_index=True)
BGUN = BGUN.query('~(韻目 in @but["韻目"])')
BGUN = pd.concat([BGUN, but],ignore_index=True)
BGUN = BGUN.fillna('-')
# 比例=1的都是沒有歧異的，所以可以用'-'替代
BGUN['比例'] = BGUN['比例'].replace(1,'-')

BGUN = pd.DataFrame(BGUN.groupby(['韻目','等', '呼', '閩南語韻母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
BGUN

比例 例字
韻目 等 呼 閩南語韻母         
之  - - i      0.15  基
       it     0.04  值
       u      0.81  思
乏  - - uat       -  乏
仙  - 合 uan       -  全
...            ... ..
麻  二 開 a      0.97  差
       e      0.03  砂
黠  - - iat       -  舌
齊  - - e      0.83  臍
       i      0.17  批

[148 rows x 2 columns]

In [20]:
# 彙整所有的聲母對應關係
BGSIANN = b_siann.query('~(聲母 in @bst["聲母"])')
BGSIANN = pd.concat([BGSIANN, bst],ignore_index=True)

BGSIANN = BGSIANN.fillna('-')
BGSIANN['比例'] = BGSIANN['比例'].replace(1,'-')

BGSIANN = pd.DataFrame(BGSIANN.groupby(['聲母','等', '閩南語聲母'])[['比例', '例字']].apply(lambda x: x.iloc[0]))
BGSIANN

比例 例字
聲母 等 閩南語聲母         
並  一 h      0.06  篷
     p      0.56  盤
     ph     0.38  磐
   三 h      0.58  縫
     p      0.35  便
...          ... ..
見  - kh     0.02  矜
     tsh    0.01  串
透  - t      0.07  踏
     th     0.93  通
邪  - s         -  松

[93 rows x 2 columns]

In [21]:
bun = D[D['文白音'] == '文']
b_tiao = bun.groupby(['閩南語調', '調']).size().reset_index(name='字數')

# 使用 transform 計算比例並新增一列 '比例'
b_tiao['比例'] = b_tiao['字數'] / b_tiao.groupby('調')['字數'].transform('sum')
b_tiao = b_tiao.sort_values(by='比例', ascending=False)
bun['temp'] = False

# 消除破音字
for i, row in b_tiao.iterrows():
    t = row['調']
    mt = row['閩南語調']

    for j in list(bun[(bun['調'] == t) & (bun['閩南語調'] == mt)]['字']):
            if bun[bun['字'] == j].sort_values(by='temp', ascending=False)['temp'].reset_index().loc[0,'temp']:
                bun = bun[~((bun['字'] == j) & (~bun['temp']))]
            else:
                bun.loc[((bun['調'] == t) & (bun['閩南語調'] == mt) & (bun['字'] == j)),'temp'] = True
        
del bun['temp']

# 重新計算比例
b_tiao = bun.groupby(['閩南語調', '調']).size().reset_index(name='字數')
b_tiao['比例'] = b_tiao['字數'] / b_tiao.groupby('閩南語調')['字數'].transform('sum')
b_tiao['比例'] = round(b_tiao['比例'], 2)
b_tiao = b_tiao.sort_values(by='比例', ascending=False)
b_tiao['例字'] = [list(bun[(bun['調'] == row['調']) & (bun['閩南語調'] == row['閩南語調'])]['字'])[0] for i, row in b_tiao.iterrows()]

# 例字
bg_tiao = pd.DataFrame(b_tiao.groupby(['閩南語調', '調', '比例'])['例字'].apply(lambda x : list(x)[0]))
bg_tiao


C:\Users\O\AppData\Local\Temp\ipykernel_10060\1865001076.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



例字
閩南語調 調 比例     
1    上 0.01  挨
     去 0.00  杆
     平 0.99  東
2    上 0.94  籠
     去 0.04  振
     平 0.02  咬
3    上 0.03  傘
     去 0.95  凍
     平 0.02  泡
4    入 1.00  樸
5    平 1.00  同
7    上 0.23  舵
     去 0.77  夢
8    入 0.99  著
     去 0.01  值

In [22]:
# 聲調圖
x = [
    b_tiao.sort_values('閩南語調')['閩南語調'],
    b_tiao.sort_values('閩南語調')['調']
]
figt = go.Figure()
figt.add_bar(x=x,y=b_tiao.sort_values('閩南語調')['比例'], text=b_tiao.sort_values('閩南語調')['例字']
            , textposition='outside', textfont=dict(size=9))

# 變更顏色、添加標題和座標軸名稱
figt.update_layout(
    colorway=['#5FA15A', '#C36EFA'],
    title='閩南語調對應的平上去入',
    xaxis_title='第N調',
    yaxis_title='比例'
)

figt.show()


In [23]:
# 圖集
figs1.show()
figs2.show()
figu1.show()
figu2.show()
figu3.show()
figt.show()

In [23]:
# 結果要用Streamlit的形式呈現
# 定義Streamlit應用程序的標題
st.title("中古音對應閩南語文音查詢系統")

# 添加輸入框，允許使用者輸入項目
st.markdown('輸入1: 中古聲母與閩南語聲母對應表')
st.markdown('輸入2: 中古聲母與閩南語聲母對應關係圖')
st.markdown('輸入3: 中古韻母與閩南語韻母對應表')
st.markdown('輸入4: 中古韻母與閩南語韻母對應關係圖')
st.markdown('輸入5: 中古韻母與閩南語韻母歧音對應聲母')
st.markdown('輸入6: 中古聲調與閩南語聲調對應關係圖')
user_input = st.text_area("請輸入要查詢的項目，以逗號分隔","")

# 定義生成圖表的函數
def generator(text):
    for i in text.split(','):
        if i == '1':
            st.markdown(BGSIANN.to_html(classes='dataframe'), unsafe_allow_html=True)
        elif i == '2':
            st.write(figs1, unsafe_allow_html=True)
            st.write(figs2, unsafe_allow_html=True)
        elif i == '3':
            st.markdown(BGUN.to_html(classes='dataframe'), unsafe_allow_html=True)
        elif i == '4':
            st.write(figu1, unsafe_allow_html=True)
            st.write(figu2, unsafe_allow_html=True)
            st.write(figu3, unsafe_allow_html=True)
        elif i == '5':
            st.markdown(BGUS.to_html(classes='dataframe'), unsafe_allow_html=True)
        elif i == '6':
            st.write(figt, unsafe_allow_html=True)
        else:
            st.write('輸入錯誤')

# 添加生成的按鈕
if st.button("查詢"):
    if user_input:
        generator(user_input)
    else:
        st.warning("請先輸入")

2023-12-27 16:20:09.385 
  command:

    streamlit run c:\Users\O\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


<br><br><br>
輸出.py檔，然後在.py檔上把所有會呈現圖表、或者是以''''''括起來的部分刪除乾淨，否則會一直跳出來
<br><br>

- - - 
#### *最後在終端機上 `F.py` 輸入 `streamlit run F.py` 就大功告成了！！*
- - - 